# Use Pretrained Model MobileNetV2 for CIFAR10 and TUNE UP

Suryasatriya Trihandaru

Data Science, FSM UKSW

## Libraries

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, Lambda, Input, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as pre
from tensorflow.keras.utils import to_categorical as hot
from tensorflow.keras.datasets import cifar10
from tensorflow.image import resize
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt

## Load CIFAR10 and Pre-Process

In [ ]:
((X,Y),(x,y))=cifar10.load_data()

170498071/170498071 [==============================] - 13s 0us/step


In [ ]:
print(X.shape)
print(Y.shape)
print(x.shape)
print(y.shape)

(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


In [ ]:
Xp = pre(X)
xp = pre(x)
Yp = hot(Y)
yp = hot(y)

In [ ]:
print(Xp.shape)
print(Yp.shape)
print(xp.shape)
print(yp.shape)

(50000, 32, 32, 3)
(50000, 10)
(10000, 32, 32, 3)
(10000, 10)


## Create Model

Use MobileNetV2 as the Pre-trained Model, set all layers untrainable. Add GlobalAveragePooling2D and output Dense. Train data.

In [ ]:
IMSIZE =224

In [ ]:
mobilnetv2=MobileNetV2(include_top=False, input_shape=(IMSIZE, IMSIZE, 3), weights='imagenet')
mobilnetv2.summary()

9406464/9406464 [==============================] - 0s 0us/step
Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                

In [ ]:
original_input=Input(shape=(32,32,3))
zoom_input=Lambda(lambda img: resize(img, (IMSIZE, IMSIZE)))(original_input)
layer = mobilnetv2(zoom_input, training=False)
layer = GlobalAveragePooling2D()(layer)
layer = Dense(256, activation='relu')(layer)
layer = Dropout(0.2)(layer)
output= Dense(10, activation='softmax')(layer)
model = Model(inputs=original_input, outputs=output)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 lambda (Lambda)             (None, 224, 224, 3)       0         
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               327936    
                                                                 
 dropout (Dropout)           (None, 256)               0     

In [ ]:
mobilnetv2.trainable=False

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 lambda (Lambda)             (None, 224, 224, 3)       0         
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               327936    
                                                                 
 dropout (Dropout)           (None, 256)               0     

Please compare the Trainable params before and after changing the status of trainable from the model.summary() above.

## First Run lr=0.001

In [ ]:
adam=Adam(0.001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
hasil=model.fit(Xp,Yp, validation_data=(xp,yp), epochs=10, batch_size=128)

Epoch 1/10
391/391 [==============================] - 74s 161ms/step - loss: 0.6064 - accuracy: 0.7939 - val_loss: 0.4950 - val_accuracy: 0.8284
Epoch 2/10
391/391 [==============================] - 62s 158ms/step - loss: 0.4395 - accuracy: 0.8467 - val_loss: 0.4380 - val_accuracy: 0.8444
Epoch 3/10
391/391 [==============================] - 62s 158ms/step - loss: 0.3905 - accuracy: 0.8640 - val_loss: 0.4230 - val_accuracy: 0.8543
Epoch 4/10
391/391 [==============================] - 62s 158ms/step - loss: 0.3603 - accuracy: 0.8751 - val_loss: 0.4175 - val_accuracy: 0.8547
Epoch 5/10
391/391 [==============================] - 62s 158ms/step - loss: 0.3300 - accuracy: 0.8857 - val_loss: 0.4136 - val_accuracy: 0.8566
Epoch 6/10
391/391 [==============================] - 62s 158ms/step - loss: 0.3053 - accuracy: 0.8928 - val_loss: 0.4051 - val_accuracy: 0.8599
Epoch 7/10
391/391 [==============================] - 62s 158ms/step - loss: 0.2878 - accuracy: 0.8977 - val_loss: 0.4065 - val_ac

## Second Run lr=0.0001

Set the last 3 layers of pretrained model trainable=True

In [ ]:
model.get_layer('mobilenetv2_1.00_224').trainable=True
for layer in model.get_layer('mobilenetv2_1.00_224').layers[:-3]:
    layer.trainable=False

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 lambda (Lambda)             (None, 224, 224, 3)       0         
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               327936    
                                                                 
 dropout (Dropout)           (None, 256)               0     

In [ ]:
adam=Adam(0.0001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
hasil2=model.fit(Xp,Yp, validation_data=(xp,yp), epochs=10, batch_size=128)

Epoch 1/10
391/391 [==============================] - 69s 164ms/step - loss: 0.1820 - accuracy: 0.9368 - val_loss: 0.4142 - val_accuracy: 0.8711
Epoch 2/10
391/391 [==============================] - 63s 161ms/step - loss: 0.1408 - accuracy: 0.9523 - val_loss: 0.4167 - val_accuracy: 0.8734
Epoch 3/10
391/391 [==============================] - 63s 162ms/step - loss: 0.1182 - accuracy: 0.9606 - val_loss: 0.4173 - val_accuracy: 0.8750
Epoch 4/10
391/391 [==============================] - 63s 162ms/step - loss: 0.0990 - accuracy: 0.9686 - val_loss: 0.4327 - val_accuracy: 0.8711
Epoch 5/10
391/391 [==============================] - 63s 162ms/step - loss: 0.0827 - accuracy: 0.9737 - val_loss: 0.4364 - val_accuracy: 0.8752
Epoch 6/10
391/391 [==============================] - 63s 161ms/step - loss: 0.0711 - accuracy: 0.9788 - val_loss: 0.4401 - val_accuracy: 0.8774
Epoch 7/10
391/391 [==============================] - 63s 161ms/step - loss: 0.0613 - accuracy: 0.9821 - val_loss: 0.4618 - val_ac

## Third Run lr=0.00001

Set the last 5 layers trainable.

In [ ]:
model.get_layer('mobilenetv2_1.00_224').trainable=True
for layer in model.get_layer('mobilenetv2_1.00_224').layers[:-5]:
    layer.trainable=False

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 lambda (Lambda)             (None, 224, 224, 3)       0         
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               327936    
                                                                 
 dropout (Dropout)           (None, 256)               0     

In [ ]:
adam=Adam(0.00001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
hasil3=model.fit(Xp,Yp, validation_data=(xp,yp), epochs=10, batch_size=64)

Epoch 1/10
782/782 [==============================] - 73s 88ms/step - loss: 0.0310 - accuracy: 0.9931 - val_loss: 0.4738 - val_accuracy: 0.8843
Epoch 2/10
782/782 [==============================] - 67s 86ms/step - loss: 0.0290 - accuracy: 0.9936 - val_loss: 0.4701 - val_accuracy: 0.8837
Epoch 3/10
782/782 [==============================] - 67s 86ms/step - loss: 0.0267 - accuracy: 0.9943 - val_loss: 0.4802 - val_accuracy: 0.8803
Epoch 4/10
782/782 [==============================] - 67s 86ms/step - loss: 0.0253 - accuracy: 0.9948 - val_loss: 0.4829 - val_accuracy: 0.8825
Epoch 5/10
782/782 [==============================] - 68s 86ms/step - loss: 0.0239 - accuracy: 0.9953 - val_loss: 0.4800 - val_accuracy: 0.8830
Epoch 6/10
782/782 [==============================] - 67s 86ms/step - loss: 0.0232 - accuracy: 0.9952 - val_loss: 0.4833 - val_accuracy: 0.8852
Epoch 7/10
782/782 [==============================] - 67s 86ms/step - loss: 0.0228 - accuracy: 0.9953 - val_loss: 0.4910 - val_accuracy:

## Fourth Run lr=0.000001

Set the last 11 layers trainable.

In [ ]:
model.get_layer('mobilenetv2_1.00_224').trainable=True
for layer in model.get_layer('mobilenetv2_1.00_224').layers[:-11]:
    layer.trainable=False
adam=Adam(0.000001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
hasil3=model.fit(Xp,Yp, validation_data=(xp,yp), epochs=10, batch_size=64)

Epoch 1/10
782/782 [==============================] - 77s 91ms/step - loss: 0.0157 - accuracy: 0.9976 - val_loss: 0.4879 - val_accuracy: 0.8869
Epoch 2/10
782/782 [==============================] - 70s 89ms/step - loss: 0.0152 - accuracy: 0.9976 - val_loss: 0.4871 - val_accuracy: 0.8874
Epoch 3/10
782/782 [==============================] - 70s 90ms/step - loss: 0.0146 - accuracy: 0.9978 - val_loss: 0.4878 - val_accuracy: 0.8876
Epoch 4/10
782/782 [==============================] - 70s 89ms/step - loss: 0.0145 - accuracy: 0.9979 - val_loss: 0.4907 - val_accuracy: 0.8871
Epoch 5/10
782/782 [==============================] - 70s 89ms/step - loss: 0.0139 - accuracy: 0.9979 - val_loss: 0.4903 - val_accuracy: 0.8860
Epoch 6/10
782/782 [==============================] - 70s 89ms/step - loss: 0.0137 - accuracy: 0.9981 - val_loss: 0.4906 - val_accuracy: 0.8874
Epoch 7/10
782/782 [==============================] - 70s 90ms/step - loss: 0.0138 - accuracy: 0.9980 - val_loss: 0.4898 - val_accuracy: